In [37]:
!pip3 install spacy 
!python3 -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [41]:
!python3 -m spacy validate



✔ Loaded compatibility table

====================== Installed models (spaCy v2.1.3) ======================
ℹ spaCy installation: /usr/local/lib/python3.6/dist-packages/spacy

TYPE      NAME             MODEL            VERSION                            
package   en-core-web-sm   en_core_web_sm   2.1.0   ✔
package   en-core-web-md   en_core_web_md   2.1.0   ✔
package   en-core-web-lg   en_core_web_lg   2.1.0   ✔
link      en               en_core_web_sm   2.1.0   ✔



In [42]:
spacy.load('en_core_web_lg')

OSError: ignored

In [39]:
import spacy
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path

from spacy.util import minibatch, compounding
from tqdm import tqdm

#LABEL = "PHONE"

# training data
TRAIN_DATA=[('Good afternoon Jasmina,  Thank you for the notice.  We made a note of it.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(129, 141, 'PHONE'), (15, 22, 'PERSON'), (90, 101, 'PERSON')]}), ('Good Afternoon Felipe,  Thank you for contacting the Client Services Center.  WORK ORDER # 14-130256  has been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(252, 270, 'EMAIL'), (325, 332, 'PERSON'), (358, 370, 'PHONE'), (15, 21, 'PERSON'), (91, 100, 'REQUEST'), (225, 237, 'PHONE')]}), ('Hello *Ruth*,  Thank you for contacting the Client Services Center with an update.  Work order number *14-130431* notes  was updated with the information below and subsequently closed.   Please contact us if there are any questions or concerns.  Regards, Rachel Client Services Center  (212)998-1001', {'entities': [(7, 11, 'PERSON'), (103, 112, 'REQUEST'), (255, 261, 'PERSON'), (286, 299, 'PHONE')]}), ('Good Afternoon Ruth,  Thank you for contacting the Client Services Center.  WORK ORDERS # 14-130429 (power issue) and 14-130431 (ac) have been submitted to the building staff for completion.  Please indicate if _*you grant permission to enter (PTE) or you prefer to admit the service personnel into your apartment.*_  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 19, 'PERSON'), (90, 99, 'REQUEST'), (118, 127, 'REQUEST'), (383, 395, 'PHONE'), (410, 428, 'EMAIL'), (483, 490, 'PERSON'), (516, 528, 'PHONE')]}), ('Good afternoon,  Please see the attached work order(s).  Thank you!  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(76, 87, 'PERSON'), (115, 127, 'PHONE')]}), ("Hi Margie,  Thank you for your message. We've updated work order *14-124630* to reflect:  836-BROADWAY,1 FL,3 FL,GRANT ACCESS TO TIAN QIU (N15255338) AND DANIEL WENCEK (N14319188) 24/7.  Please let us know if you have any questions.  Best,  Eric Vokral Client Services Center (212) 998-1056 - Direct (212) 998-1001 - Office", {'entities': [(3, 9, 'PERSON'), (66, 75, 'REQUEST'), (90, 112, 'FAC'), (129, 137, 'PERSON'), (154, 167, 'PERSON'), (241, 252, 'PERSON'), (276, 291, 'PHONE'), (300, 314, 'PHONE')]}), ('Good Evening Margie,  Your request to adjust access for *TIAN QIU (N15255338) AND DANIEL WENCEK (N14319188) FROM 8:45AM - 5:30PM to *7:45 A.M. - 5:30 P.M Monday through Friday generated the following response from Card Services:  The choice for that location is either 9am-6pm, M-F or 24/7.  Please advise.  *14-124630** 836-BROADWAY,1 FL,3 FL,GRANT ACCESS TO TIAN QIU (N15255338) AND DANIEL WENCEK (N14319188) FROM 8:45AM - 5:30PM MONDAY THROUGH FRIDAY*  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001    Dear Vanessa,  I need to make a slight change to work order 14-124630 (below).  Daniel Wencek (N14319188) needs access to 838 Broadway from 7:45 A.M. - 5:30 P.M. Can you make this change?  Many thanks.', {'entities': [(13, 19, 'PERSON'), (57, 65, 'PERSON'), (82, 95, 'PERSON'), (309, 318, 'REQUEST'), (321, 343, 'FAC'), (360, 368, 'PERSON'), (385, 399, 'PERSON'), (516, 523, 'PERSON'), (549, 561, 'PHONE'), (570, 577, 'PERSON'), (645, 658, 'PERSON'), (687, 700, 'FAC')]}), ('Good afternoon all,  I received a call from Mr. Watson in 100-Bleecker 8B.  He wanted to make sure that the long vertical strip windows that are present in several dining rooms in the Silver Towers apts will be cleaned in addition to the rest.  He noted that last year these were not cleaned.  His contact number is 646-467-2316.  *Work Order # 14-130289* *  Description:* 100-BLEECKER ST,8 FL,APT 8B,TENANT HAS WINDOW CLEANING INQUIRY  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(44, 54, 'PERSON'), (58, 73, 'FAC'), (184, 197, 'FAC'), (316, 328, 'PHONE'), (345, 354, 'REQUEST'), (373, 400, 'FAC'), (518, 531, 'PERSON'), (555, 569, 'PHONE')]}), ('Good afternoon Victor,  We submitted work order *14-129000* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (49, 58, 'REQUEST'), (242, 257, 'PERSON'), (280, 293, 'PHONE')]}), ('Good afternoon Kevin,  Thank you for contacting the Client Services Center. The following work order has been submitted to Card Services as indicated by your request below:  *14-128888* 14-E 78TH ST,1 FL,LOBBY,PROVIDE INDEFINITE FRONT DOOR SWIPE ACCESS TO SARAH BARACK, N12608386, SB340 FOR THE FOLLOWING TIMES: M-F 8A-10P, SA 9A-9P, SU 9A-9P.  Please let us know if you have any questions.  Best,  Eric Vokral Client Services Center (212) 998-1056 - Direct (212) 998-1001 - Office', {'entities': [(14, 20, 'PERSON'), (175, 184, 'REQUEST'), (186, 203, 'FAC'), (256, 268, 'PERSON'), (399, 410, 'PERSON'), (434, 448, 'PHONE'), (458, 472, 'PHONE')]}), ('Good afternoon Adam,  Thank you for clarifying. We submitted work order *14-129324* to building staff, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (73, 82, 'REQUEST'), (297, 308, 'PERSON'), (336, 348, 'PHONE')]}), ('Good afternoon Randy,  Thank you for contacting the Client Services Center.  Please note that your setup request has been forwarded to the Facility Manager for approval. Once approval is received, we will send you an additional email, issuing a work order that details the work to be conducted.  Please feel free to contact us if you have any questions or concerns.  Best,  Seamus Mullin Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (374, 387, 'PERSON'), (411, 423, 'PHONE')]}), ('Good Afternoon Peter,  Thank you for contacting the Client Services Center.  WORK ORDER # 14-128992  has been submitted to the Utility Shop for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (90, 100, 'REQUEST'), (222, 234, 'PHONE'), (249, 267, 'EMAIL'), (322, 329, 'PERSON'), (355, 367, 'PHONE')]}), ('Good Afternoon Nursing Operations,  Thank you for contacting the Client Services Center.  WORK ORDER # 14-130107  has been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(103, 112, 'REQUEST'), (237, 249, 'PHONE'), (264, 282, 'EMAIL'), (337, 344, 'PERSON'), (370, 382, 'PHONE')]}), ('Good afternoon Peter,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to the building engineers for completion, as indicated by your request below.  *Work Order # 14-129593* *  Description:* JACOBS ACADEMIC,5 FL,RMS 574-577,NOT ENOUGH AIR,CONFIRMED WITH ENGINEER PAUL  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (239, 248, 'REQUEST'), (267, 299, 'FAC'), (339, 343, 'PERSON'), (426, 439, 'PERSON'), (463, 477, 'PHONE')]}), ('Good afternoon Roger,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to the CBS staff for completion, as indicated by your request below.  *Work Order # 14-130239* *  Description:* 10-ASTOR PL,5 FL,RM 504G,VACUUM ROOM  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (230, 239, 'REQUEST'), (258, 294, 'FAC'), (377, 390, 'PERSON'), (414, 428, 'PHONE')]}), ('Hello Raymond,  Please be advised of the work order below:  Description                    105-E 17TH ST,3 FL,ROOM 329,DOOR WILL NOT LOCK.  Requester                      VERONICA  Phone                             82848  Thank you in advance for all of your help.  Regards, Rachel Client Services Center', {'entities': [(6, 13, 'PERSON'), (91, 118, 'FAC'), (275, 281, 'PERSON'), (149, 179, 'PERSON')]}), ('Good afternoon Hilary,  My apologies for the delay in getting back to you about the chargebacks. Radhica was able to process the payment to the chartfield provided.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(211, 223, 'PHONE'), (15, 21, 'PERSON'), (97, 104, 'PERSON'), (172, 187, 'PERSON')]}), ('Good afternoon Hilary,  I have been informed by Brent that as per your conversation with him, he has followed up and he will be in touch with you as soon as he hears back.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(258, 273, 'PERSON'), (297, 309, 'PHONE'), (15, 21, 'PERSON'), (48, 54, 'PERSON')]}), ('Good afternoon Hilary,  I followed up with Radhica regarding the charge back for your Dependable move and let her know the account needs to be charged as soon as possible. I will let you know as soon as I receive a response from her.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(43, 50, 'PERSON'), (241, 256, 'PERSON'), (280, 294, 'PHONE'), (15, 21, 'PERSON')]}), ("Good afternoon Hilary,  Thank you for this update. We've asked finance to process your request again with this chartfield.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 21, 'PERSON'), (192, 207, 'PERSON'), (231, 243, 'PHONE')]}), ('Good afternoon Hilary,  I hope this email finds you well.  Unfortunately, we could not process payment for the following work order as chartfield *63130-25-74100-F7018* was invalid. * **14-023429** **719-BROADWAY,12 FL,MOVE ON FRI 10/25 AT 11AM, PICK UP AV EQUIPMENT FROM DIGITAS AT 355 PARK AVE SOUTH, 12 FL AND BRING TO MICHAEL RANIERI, SEE EXTRA DESCRIPTION CHARTIFELD 63130-25-74100-F7018 **REF 3047 SPEAK TO NELLIE FOR DETAILS* *Cost: $658.50** * Please provide the correct chartfield at your earliest convenience so we can process our journal entries.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (186, 195, 'REQUEST'), (200, 218, 'FAC'), (227, 244, 'DATE'), (283, 308, 'FAC'), (322, 337, 'PERSON'), (413, 419, 'PERSON'), (565, 580, 'PERSON'), (604, 616, 'PHONE')]}), ('Good afternoon Matt,  Thank you for clarifying. We submitted work order *14-129861* to the Locksmith Shop, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (73, 82, 'REQUEST'), (301, 312, 'PERSON'), (340, 352, 'PHONE')]}), ('Good afternoon David,  Thank you for contacting the Client Services Center. We submitted work order #:**14-129121 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (104, 113, 'REQUEST'), (312, 327, 'PERSON'), (351, 363, 'PHONE')]}), ('Good afternoon David,  Thank you for contacting the Client Services Center. We submitted work order #:**14-129046 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (104, 113, 'REQUEST'), (312, 327, 'PERSON'), (351, 363, 'PHONE')]}), ('Good Afternoon Amber,  WORK ORDER # 14-130493 now includes a request for an estimate.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (36, 45, 'REQUEST'), (180, 192, 'PHONE')]}), ('Good Afternoon Amber,  WORK ORDER # 14-130493 has been submitted to the Paint Shop for completion.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (36, 45, 'REQUEST'), (160, 167, 'PERSON'), (193, 205, 'PHONE')]}), ('Good Morning Luis,  Please see the below request from Ariel who has followed up several times regarding the below questions and advise.  Please contact Ariel via email at ariel.assaf@gmail.com or phone at 617-407-2825  to discuss.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(13, 17, 'PERSON'), (54, 59, 'PERSON'), (152, 157, 'PERSON'), (205, 217, 'PHONE'), (171, 192, 'EMAIL'), (292, 299, 'PERSON'), (325, 337, 'PHONE')]}), ('Good afternoon Eric,  Thank you! I will forward the work order accordingly.  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (84, 95, 'PERSON'), (123, 135, 'PHONE')]}), ('Good afternoon Eric,  Thank you for this update.  The below information has been added into the work order notes log.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (186, 202, 'PERSON'), (226, 238, 'PHONE')]}), ('Good afternoon CBS,  This client has followed up a few times since May about this light out. Is this something that needs to be forwarded to the operators?  Thanks for your help!  Best,  James Walsh NYU Client Services Center 212-998-1001   -------- Original Message -------- Subject: \tRe: Fwd: THIRD REQUEST: Work Request Number: 14-103939 Date: \tTue, 1 Jul 2014 10:47:29 -0400', {'entities': [(187, 198, 'PERSON'), (226, 238, 'PHONE'), (331, 340, 'REQUEST'), (348, 378, 'DATE')]}), ('Good afternoon Nicolas,  Thank you for this update.  The work order has been closed and the below information has been added into the notes log.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (214, 229, 'PERSON'), (253, 265, 'PHONE')]}), ('Good afternoon Maryanne,  Thank you for contacting the Client Services Center. Please note that the following work orders have been generated and sent to Card Services for completion, as indicated by your request below.  *Work Order # 14-129016* * Description:* 105-E 17TH ST,4 FL,ASSIGN ACCESS TO LINDSAY NICK (NET ID LN32) UNIV ID N18979468, MON-FRI 9AM-8PM *Work Order # 14-129021* * Description:* 194-MERCER ST,6 FL,ASSIGN ACCESS TO MIAO-SHAN (EVA) LIN (NET ID MSL471) UNIV ID N13428591, MON-FRI 9AM-8PM  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(14, 23, 'PERSON'), (235, 244, 'REQUEST'), (262, 280, 'FAC'), (298, 310, 'PERSON'), (344, 359, 'DATE'), (374, 383, 'REQUEST'), (401, 419, 'FAC'), (437, 456, 'PERSON'), (492, 507, 'DATE'), (590, 603, 'PERSON'), (627, 641, 'PHONE')]}), ('Good afternoon Carbie,  Thank you for contacting the Client Services Center. We submitted work order *14-129771 *as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (102, 111, 'REQUEST'), (295, 310, 'PERSON'), (334, 346, 'PHONE')]}), ('Good afternoon Marilyn,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to building staff for them to assess and repair on a sooner timeframe if possible.  *Work Order # 14-130337* *  Description:* 3-WASH SQ V,6 FL,APT 6H,DINING ROOM,CRACKED WINDOW PANE  _Please advise whether you provide Permission To Enter (PTE) or if you would prefer to admit the building service personnel (TWA)._  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 22, 'PERSON'), (246, 255, 'REQUEST'), (274, 297, 'FAC'), (545, 558, 'PERSON'), (582, 596, 'PHONE')]}), ('Good afternoon Jim,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to the CBS staff for completion, as indicated by your request below.  *Work Order # 14-129066* *  Description:* OTHMER HALL,5-8 FLS,MULTIPLE RMS,LOCATE AND REPLACE DAMAGED/MISSING FURNITURE AS PER LIST FROM CJ BARNES IN EXTRA DESCRIPTION, MUST BE COMPLETED BY TOMORROW 6/28  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 18, 'PERSON'), (413, 417, 'DATE'), (500, 513, 'PERSON'), (537, 551, 'PHONE'), (228, 237, 'REQUEST'), (256, 275, 'FAC'), (351, 360, 'PERSON')]}), ('Good afternoon Jasmina and all,  Please accept my apologies for the error.  The proper work order description is below:  *Work Order # 14-130317**  Description:* 1-WASH SQ N,1 FL,RM 109,ELECTRICIANS TO INSTALL TWO NEW PENDANT FIXTURES AND REMOVE OLD ONES   Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(135, 144, 'REQUEST'), (162, 185, 'FAC'), (338, 351, 'PERSON'), (375, 389, 'PHONE'), (15, 22, 'PERSON')]}), ("Good afternoon,  Per James' 12:30pm email, a work order is not required as access has been assigned.  Work order *14-128888 *has been canceled.  Best,  Eric Vokral Client Services Center (212) 998-1056 - Direct (212) 998-1001 - Office", {'entities': [(114, 123, 'REQUEST'), (152, 163, 'PERSON'), (211, 225, 'PHONE'), (21, 26, 'PERSON'), (187, 201, 'PHONE')]}), ('Good afternoon Richard,  Thank you for following up with our office. Your work order has been confirmed by Action carting and will be completed on the specified dates.  *In the future, please send all CSC correspondence to our email alias contactcsc@nyu.edu.*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(329, 344, 'PERSON'), (15, 22, 'PERSON'), (239, 257, 'EMAIL'), (368, 380, 'PHONE')]}), ('Good afternoon Richard,  Thank you for contacting the Client Services Center. We submitted work order *14-128076* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(296, 311, 'PERSON'), (335, 348, 'PHONE'), (15, 23, 'PERSON'), (103, 112, 'REQUEST')]}), ("Good afternoon Eric,  I have redirected the work order as you've asked.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001", {'entities': [(197, 209, 'PHONE'), (15, 19, 'PERSON'), (158, 173, 'PERSON')]}), ('Hello Edny,  Good afternoon.  Thank you, we will close the work orders as quickly as possible.  Regards, Brent Maddox Client Services Center (212)998-1001', {'entities': [(141, 154, 'PHONE'), (6, 10, 'PERSON'), (105, 117, 'PERSON')]}), ('Good afternoon all,  Please be advises of the below work order submitted by Erik Cruz  *Work Order # 14-128889* *  Description:* MEYER,BSMT,ASSIGN CNS GENERAL AND ANIMAL ACCESS FOR JOSEPH LEBOWITZ, JZL215, N14385610, *151527, 24/7 INDEFINITELY  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(101, 110, 'REQUEST'), (326, 339, 'PERSON'), (76, 85, 'PERSON'), (129, 140, 'FAC'), (181, 196, 'PERSON'), (362, 377, 'PHONE')]}), ('Good morning Ingrid,  Thank you for contacting the Client Services Center. We submitted work order #*14-129416 ***as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(13, 19, 'PERSON'), (101, 110, 'REQUEST'), (374, 393, 'EMAIL'), (297, 312, 'PERSON')]}), ('Good afternoon Jasmina,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to the Electricians for completion, as indicated by your request below.  *Work Order # 14-130315* *  Description:* FAIRCHILD,1 FL,RM 109,ELECTRICIANS TO INSTALL TWO NEW PENDANT FIXTURES AND REMOVE OLD ONES  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 22, 'PERSON'), (235, 244, 'REQUEST'), (436, 449, 'PERSON'), (263, 284, 'FAC'), (473, 487, 'PHONE')]}), ('Good afternoon Bill,  Please advise on the below follow up from Maria on her set up in Main 208.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001  -------- Original Message -------- Subject: \tRe: tables for an event in 208 Silver, Monday, 6/30 from 5:00-8:30 pm Date: \tFri, 27 Jun 2014 13:17:41 -0400', {'entities': [(15, 19, 'PERSON'), (64, 69, 'PERSON'), (183, 198, 'PERSON'), (87, 95, 'FAC'), (222, 234, 'PHONE'), (308, 318, 'FAC'), (320, 389, 'DATE')]}), ("Good morning Everyone,  I hope this message finds you well. Please note that the following work order has been generated for 10 Washington Place. Attached is a copy of the work order.     *Work Order* *_#14-130161 _*    *Description*: 10-WASH PL,1 & 6 FL,BUILDING ENTRANCE DOOR & ROOMS    607 &608,PROVIDE 24/7 CARD ACCESS TO ESTI BLANCO (N10859207; EB134)    FROM TODAY 7/1 UNTIL 8/5     *Work Order* *_#14-130158 _*    *Description*: 10-WASH PL,1 & 6 FL,BUILDING ENTRANCE DOOR & ROOMS    607 &608,PROVIDE 24/7 CARD ACCESS TO LAURA GWILLIAMS (N12421217;    LEG5) FROM TODAY 7/1 UNTIL 8/5  If needed, here is the client's contact information.     *Name:*  Theresa Leung    *Phone: *87951*    **Email: *Teresa.Leung@nyu.edu  Please feel free to contact us if you have any questions or concerns.  Best Wishes, Kerri Campbell  NYU Client Services Center (212) 998-1001", {'entities': [(204, 213, 'REQUEST'), (235, 254, 'FAC'), (436, 455, 'FAC'), (527, 542, 'PERSON'), (575, 578, 'DATE'), (585, 589, 'DATE'), (656, 669, 'PERSON'), (702, 722, 'EMAIL'), (808, 822, 'PERSON'), (125, 144, 'FAC'), (326, 336, 'PERSON'), (371, 374, 'DATE'), (381, 384, 'DATE'), (405, 414, 'REQUEST'), (851, 865, 'PHONE')]}), ('Hello George,  Good morning.  The Gardening Shop phase of the work order is closed.  We are currently waiting to close phase 002 for the Carpentry Shop.  Once we receive the work order from the Carpentry Shop we will be able to close the entire work order.  Please let me know if there are any questions.  Thank you.  Regards, Brent Maddox Client Services Center (212)998-1001', {'entities': [(327, 339, 'PERSON'), (6, 12, 'PERSON'), (363, 376, 'PHONE')]}), ('Hello George,  Please be advised that the work order was closed out.  Regards,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(80, 94, 'PERSON'), (6, 12, 'PERSON'), (118, 130, 'PHONE')]}), ('Good morning Gardening Shop,  I hope this message finds you well. Please note that the following work order has been generated for the 1st floor of Bobst Library. Attached is a copy of the work order. Please confirm your receipt of this request at your earliest convenience.     *Work Order* _#14-109512.003 _    *Description*: BOBST LIBRARY,1 FL,SCHWARTZ PLAZA (BETWEEN BOBST AND    50-W 4TH ST),REPAIR/LOOSE BROKEN COBBLESTONE. *ATTN MIKE: AS PER    CHRIS - GARDEN SHOP TO ASSIST IN REPAIR OF PAVERS  Please feel free to contact us if you have any questions or concerns.  Thank you for your time and assistance, Kerri Campbell  Client Services Center (212) 998-1001', {'entities': [(328, 395, 'FAC'), (294, 307, 'REQUEST'), (436, 440, 'PERSON'), (452, 457, 'PERSON'), (614, 628, 'PERSON'), (653, 667, 'PHONE')]}), ('Good evening  James,  We have just received an update in response to your inquiry from the property managers.  /On July 8th, we will be sending email to all WSV tenants advising them of the measurement schedule. We are setting up a Google link response for tenants to note their approval to enter (or not). FYI- measuring starts on July 27/28 and is done early Sept./  Please feel free to contact us if you have any questions or concerns and we will be happy to follow up as needed.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(14, 19, 'PERSON'), (115, 123, 'DATE'), (332, 342, 'DATE'), (361, 367, 'DATE'), (494, 507, 'PERSON'), (531, 545, 'PHONE')]}), ('Good afternoon James,  Thank you for contacting the Client Services Center. Please note that we have passed on your inquiry to the building management staff.  We are not sure at this point if a work order will be needed to track your preference.  We will let you know as soon as we receive a response.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (384, 397, 'PERSON'), (421, 435, 'PHONE')]}), ('Good afternoon Laurel,  Thank you for contacting the Client Services Center. Please note we reached out to our pest control company who informed us Kimmel Center is scheduled every Tuesday. They have also informed us that someone should be onsite today to treat for this issue.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (148, 161, 'FAC'), (181, 188, 'DATE'), (325, 337, 'PHONE'), (286, 301, 'PERSON')]}), ('Good afternoon Monroe,  Thank you for contacting our office and again we apologize for this delay and inconvenience. We have added your note to the Work Order and are now awaiting advisement from the Facility Manager with regards to any additional steps to be taken to assist in achieving a resolution. We have requested that AKA treatment take place as soon as possible and are requesting more long term solutions for this matter. Once we receive any updates, we will contact you with the information and continue to monitor this issue. Once more, we apologize for the ongoing nature of this issue and the severity of the matter.  In the meantime, please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 21, 'PERSON'), (715, 731, 'PERSON'), (754, 766, 'PHONE')]}), ('Good afternoon David,  I hope this message finds you well. Please advise if visits could be scheduled on a monthly or bi-monthly bases.    Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail  -------- Original Message -------- Subject: \tRe: confirmation Re: Exterminatior Date: \tMon, 7 Jul 2014 14:47:20 -0400', {'entities': [(15, 20, 'PERSON'), (146, 161, 'PERSON'), (185, 197, 'PHONE'), (321, 358, 'DATE')]}), ('Good afternoon Laurel,  Thank you for contacting the Client Services Center. We deeply apologize for the this on going issue and inconvenience this has caused. Please note we have rescheduled the pest control company to exterminate the whole 8th floor as well as room 806. Please be advised work order _*14-125361*_ has been updated as per your email below. We have also forwarded your email to the Facility manager for advise if scheduling could be done monthly or bi-monthly.  Please do not hesitate to reach out to us if you have any questions or concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (242, 252, 'FAC'), (263, 271, 'FAC'), (304, 313, 'REQUEST'), (569, 584, 'PERSON'), (646, 664, 'EMAIL')]}), ('Good Morning Laurel,  The exterminator has been notified and a tech will visit your location today (no ETA provided).  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(13, 19, 'PERSON'), (179, 186, 'PERSON'), (212, 224, 'PHONE')]}), ('Good afternoon Laurel,  Thank you for contacting the Client Services Center. We submitted work order *14-125361 *as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (102, 111, 'REQUEST'), (295, 310, 'PERSON'), (334, 347, 'PHONE')]}), ('Hello Ariel,  Thank you for contacting the Client Services Center. Please be advised that the work order was updated with the date below.  Please let us know if you have any other questions/concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 11, 'PERSON'), (208, 222, 'PERSON'), (246, 258, 'PHONE')]}), ('Good Morning Ariel,  Thank you for contacting the Client Services Center.  Please pardon the delay and for any inconvenience this may have caused.  Your request has been forwarded to the faculty housing management and building staff for review and advisement.  Once a response has been received, you will be notified.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(13, 18, 'PERSON'), (384, 396, 'PHONE'), (411, 429, 'EMAIL'), (484, 491, 'PERSON'), (517, 529, 'PHONE')]}), ('Good afternoon Ariel,  Thanks for following up with us and we apologize for the delay.  I have reached out again to the facility manager and building staff, asking them for an expedited response to your inquiries.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (296, 309, 'PERSON'), (333, 347, 'PHONE')]}), ('Good afternoon Ariel,  I have forwarded your inquiries to building personnel for advisement. We will contact you as soon as we receive a response.  Thank you, and please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (243, 254, 'PERSON'), (282, 294, 'PHONE')]}), ("Good afternoon Ariel,  We've forwarded your email to the building staff for advisement. We will contact you as soon as we receive a response.  Please contact us if work is not completed or there are any questions or concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 20, 'PERSON'), (272, 284, 'PHONE'), (233, 248, 'PERSON')]}), ('Good afternoon Ariel,  Thank you for contacting the Client Services Center. We generated the below work order, as per your request. Please note that elevator reservations are only available from 9:00AM-5:00PM.  *Work Order:* *14-117416* *Description: *100-BLEECKER ST,13 FL,APT 13B,ELEVATOR RESERVATION FOR EVENING OF FRIDAY AUG 15 (UNTIL 5PM)  Please also note that use of the elevator may not be exclusive (other tenants may be moving in/out, maintenance staff may be transporting items, etc.).  Please feel free to contact us if you have any questions or concerns.  Best regards,  James Walsh NYU Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (226, 235, 'REQUEST'), (252, 281, 'FAC'), (623, 637, 'PHONE'), (584, 595, 'PERSON')]}), ('Good afternoon Bill,  Thank you for the approval. Work order *14-128933 *has been generated for this request.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (62, 71, 'REQUEST'), (179, 195, 'PERSON'), (218, 230, 'PHONE')]}), ('Good afternoon David,  Thank you for providing us with that information. Please note work order _*14-129234*_ has been updated accordingly.  Please let us know if you have any other questions/concerns.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(14, 20, 'PERSON'), (98, 107, 'REQUEST'), (210, 225, 'PERSON'), (249, 261, 'PHONE')]}), ('Hello Tom,  Thank you for contacting the Client Services Center. Please be advised that I have forward your requests to the facility manager for advisement. We will let you know as soon as we receive a response.  In the mean time, please feel free to contact us with any questions or concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 9, 'PERSON'), (302, 316, 'PERSON'), (340, 352, 'PHONE')]}), ('Good afternoon all,  Please be advised of the below work order submitted by Tina Coye:  *Work Order # 14-130439* *  Description:* 3-WASH SQ V,1 FL,APT 1M,ASSIGN ACCESS TO NICOLAS HEPPDING, N19843580, NAH293, 24/7, INDEFINITELY  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(76, 85, 'PERSON'), (102, 111, 'REQUEST')]}), ('Good afternoon Tyshawn,  Thank you for contacting the Client Services Center. We submitted work order *14-129042* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (103, 112, 'REQUEST'), (296, 311, 'PERSON'), (335, 347, 'PHONE')]}), ("Good evening Jim,  Please be advised we received the below note from the engineer Barney in regards to the PM below.  It has been marked as completed and closed out.  *Work Order # 14-115992* *  Description:* MONTHLY PORTABLE FIRE EXTINGUISHER INSPECTION - ACADEMIC - ENTIRE BUILDING *PERFORMED BY FCM/CONTRACTOR STAFF *REQUIRED BY NFPA 10 - NYCFD *THE MONTHLY INSPECTION MUST DETAIL THE VISUAL CONDITION OF THE FIRE EXTINGUISHER  *CLOSING*: FE BY FREIGHT NEEDS TO BE HUNG. FE BY PLUMBING SUPPLIES NEEDS TO BE HUNG. NO FE IN JB 555. FOUND ONLY 1 FE ON 4 FL SOUTH SIDE HALLWAY COMPLETELY BLOCKED. NO FE'S ON 3 FL COMMON AREA - ONLY ONE FE FOUND AT ALL IN 341 AND TAG WAS EXPIRED IN 2013 7/2013. ONLY 1 FE ON 2ND FL ALSO. BARNEY GUZZO 06-12-14 0130-0300  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001", {'entities': [(12, 16, 'PERSON'), (82, 88, 'PERSON'), (181, 190, 'REQUEST'), (720, 732, 'PERSON'), (733, 741, 'DATE'), (871, 885, 'PHONE'), (834, 847, 'PERSON')]}), ('Good afternoon Nichole,  Thank you for contacting the Client Services Center. We submitted work order *14-128811* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (103, 112, 'REQUEST'), (296, 311, 'PERSON'), (335, 347, 'PHONE')]}), ('Good afternoon John,  Thank you for contacting the Client Services Center. We submitted work order *14-130480* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (100, 109, 'REQUEST'), (293, 308, 'PERSON'), (332, 344, 'PHONE')]}), ('Good afternoon Matt and Nick,  Please disregard the below email, the incorrect email and accidentally forwarded on.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 20, 'PERSON'), (24, 28, 'PERSON'), (123, 138, 'PERSON'), (162, 174, 'PHONE')]}), ('Good afternoon Matt and Nick,  Please see the below request and let us know if there are any updates to provide the client.  *14-122867** * *2-METROTECH,8 FL,RM 874,CUT (1) COPY OF KEY, CYLINDER NUMBER AB27, CHARTFIELD 63270-11-63252, INFORM CLIENT IF KEY CAN BE SENT VIA INTER-OFFICE ENVELOPE** **Contact    ERIC MAIELLO** **Contact Email    eric.maiello@nyu.edu*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail  -------- Original Message -------- Subject: \tRe: Confirmation Re: Spare Keys / RUSH Date: \tFri, 27 Jun 2014 12:43:34 -0400', {'entities': [(15, 20, 'PERSON'), (24, 28, 'PERSON'), (126, 135, 'REQUEST'), (141, 164, 'FAC'), (309, 321, 'PERSON'), (343, 363, 'EMAIL'), (434, 450, 'PERSON'), (473, 485, 'PHONE'), (619, 650, 'DATE')]}), ('Good afternoon Eric,  Thank you for contacting the Client Services Center. We submitted work order *14-128897***as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (100, 109, 'REQUEST'), (294, 309, 'PERSON'), (333, 345, 'PHONE')]}), ('Hello Ed and all,  Thanks for all of your help today, and I hope you feel much better soon.  I reached out to the client Inez who explained that it was fine if the panel was downloaded tomorrow morning, as long as the reader would be ready by the time they arrived for work at 9am.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(6, 8, 'PERSON'), (120, 125, 'PERSON'), (364, 377, 'PERSON'), (401, 415, 'PHONE')]}), ('Good Morning James,  Thanks for the update.  The client will be informed.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(168, 180, 'PHONE'), (13, 18, 'PERSON'), (135, 142, 'PERSON')]}), ('Good Afternoon Marilyn,  WORK ORDER # 14-130337 has been updated and the building staff has been notified.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 22, 'PERSON'), (38, 47, 'REQUEST'), (166, 175, 'PERSON'), (201, 213, 'PHONE')]}), ("Good afternoon Michael,  Thank you for contacting the Client Services Center. We submitted work order #:**14-128784 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Also please let us know if you give the maintenance workers PTE (permission to enter) if you're not home or TWA (You will admit).  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001", {'entities': [(15, 22, 'PERSON'), (106, 115, 'REQUEST'), (445, 460, 'PERSON'), (484, 496, 'PHONE')]}), ('Good Afternoon Cynthia,  Thank you for contacting the Client Services Center.  We apologize for the delay in service and any inconvenience it may have caused.  The below work orders have been updated for and have been resubmitted to the facility service personnel to investigate and repair.  We have contacted them via email to expedite.  Please reference these numbers if follow up is required.  Please contact us if work is not completed or there are any questions or concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (526, 538, 'PHONE'), (487, 502, 'PERSON')]}), ('Good Afternoon Cynthia,  Thank you for contacting the Client Services Center.  WORK ORDERS (see below) have been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 22, 'PERSON'), (254, 272, 'EMAIL'), (360, 372, 'PHONE'), (227, 239, 'PHONE'), (327, 334, 'PERSON')]}), ('-------- Original Message -------- Subject: \tRe: [Fwd: Request for theater/room rental Pfizer Theater] Date: \tTue, 1 Jul 2014 13:42:37 -0400', {'entities': [(110, 140, 'DATE')]}), ("Good afternoon Roger,  We've contacted CBS to have your request expedited. They will send someone over as soon as they are available. We will contact you if there are any updates.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 20, 'PERSON'), (248, 264, 'PERSON'), (288, 300, 'PHONE')]}), ('Good afternoon David,  We submitted work order*14-128743* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(47, 56, 'REQUEST'), (15, 20, 'PERSON'), (240, 255, 'PERSON'), (279, 291, 'PHONE')]}), ('Good Afternoon Kenny,  Thanks for the information.  Please inform CSC when a work order is required.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(14, 20, 'PERSON'), (162, 169, 'PERSON'), (195, 207, 'PHONE')]}), ('Good evening Kiran,  Thanks for following up with us and providing the chartfield.  I have noted work order *14-096022* accordingly.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 18, 'PERSON'), (109, 118, 'REQUEST'), (215, 228, 'PERSON')]}), ('Good afternoon David,  Thank you for your quick response and information provided. We have reached out to Helen and requested assistance with this matter. Pending her response, we will contact the client as needed.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (242, 257, 'PERSON'), (281, 293, 'PHONE'), (106, 111, 'PERSON')]}), ('Good afternoon Helen,  I hope you are well! Per Dave Critedes, we were asked to contact you for advisement with regards to the below request for information on rates related to scheduling a guard for a new Lab location. The client\'s contact information is Adam Cragg (adam.cragg@nyu.edu).  Our office received the below follow up from a client who wishes to know the following: /"How much it would cost to have a security guard in the evenings. For your reference, we are part of the Provost\'s office and our lab is a resource for all current NYU students and faculty. / / / // /Can you please give a quote for having a security guard at 16 Washington Place from 5pm-9pm, Monday-Friday for the school year? Also, if it\'s possible to have some details about the costs for one-off events, that would be very helpful as well; minimum times, rate per hour, etc."  /Please advise on what information we can provide the client or if you require any additional formation in order to process this inquiry. * ***Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001   -------- Original Message -------- Subject: \tRe: [*Please Advise* - Pending Request] Re: Security Officer Date: \tThu, 3 Jul 2014 16:00:57 -0400', {'entities': [(15, 20, 'PERSON'), (48, 61, 'PERSON'), (256, 266, 'PERSON'), (268, 286, 'EMAIL'), (638, 685, 'FAC'), (1117, 1133, 'PERSON'), (1156, 1169, 'DATE'), (1284, 1314, 'DATE')]}), ('Good afternoon Adam,  Thanks for continuing to follow up with us, and please accept our apologies for any delay.  Unfortunately we are not able to give an estimated time as each request is different and may take a different amount of time for Public Safety to research and get back to us.  However, we will continue to follow up with Public Safety, and please follow up with us as well.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 19, 'PERSON'), (467, 482, 'PERSON'), (506, 520, 'PHONE')]}), ('Good afternoon Adam,  Thank you for contacting our office regarding this matter. We have forwarded your inquiry to the department within the Public Safety office which provides us with estimate information. Once we receive their response with regards to costs for specific and more general coverage, we will contact you with this information.  In the meantime, please contact our office if you have any questions or concerns regarding this matter.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (456, 471, 'PERSON'), (495, 507, 'PHONE')]}), ("Good afternoon Raymond,  We submitted work order *14-128916* as per your request. You may refer to this number if you have any questions in the future regarding this matter. We've also forwarded your email to Public Safety for their advisement.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 22, 'PERSON'), (50, 59, 'REQUEST'), (314, 329, 'PERSON'), (353, 366, 'PHONE')]}), ("Hi Rich,  Thank you for your message. We'll close these as quickly as possible.  Please let us know if you have any questions.  Best,  Eric Vokral Client Services Center (212) 998-1056 - Direct (212) 998-1001 - Office", {'entities': [(3, 7, 'PERSON'), (135, 146, 'PERSON'), (170, 184, 'PHONE'), (194, 208, 'PHONE')]}), ("Good afternoon Raymond,  We received the below e-mail from Fara Ortiz requesting a status update on the below work order. Please advise what you would like us to tell the client.  *Work Order# 14-122468* *Description:* 105-E 17TH ST,2 FL,REGISTRAR'S OFFICE,CABINET DRAWER NEEDS A SCREW REPLACED  Please feel free to contact us if you have any questions or concerns.  Best Wishes, Eric H. Emma Client Services Center (212) 998-1001    -------- Original Message -------- Subject: \tRe: CONFIRMATION Re: Cabinet Drawer Repair, Registrar Office, 105 East 17th Street, 2nd Floor Date: \tFri, 27 Jun 2014 10:42:58 -0400", {'entities': [(15, 22, 'PERSON'), (59, 69, 'PERSON'), (192, 202, 'REQUEST'), (219, 256, 'FAC'), (380, 392, 'PERSON'), (416, 430, 'PHONE'), (523, 573, 'FAC'), (580, 611, 'DATE')]}), ('Good afternoon All,  The below request has not been completed as per the client  /14-126761// //KIMMEL CENTER, MAIN ENTRANCE & 4TH FL,SOLARIUM COMPUTER LAB, RM 409,AND 75 THIRD AVE, C3 COMPUTER LAB// //PLEASE PROVIDE CARD ACCESS TO:// //WILLIAM PRIDE - CONTRACTOR// //N17567133// //WP575// //INDEFINITE// //24/7// / /Contact    TAMARA SANTIAGO// //Contact Phone    212.998.1209// //Contact Email    ts63@nyu.edu/  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail  -------- Original Message -------- Subject: \tRe: CONFIRMATION Re: Please grant swipe access to William Pride _ N17567133 Date: \tFri, 27 Jun 2014 09:53:34 -0400', {'entities': [(82, 91, 'REQUEST'), (96, 197, 'FAC'), (237, 250, 'PERSON'), (328, 343, 'PERSON'), (365, 377, 'PHONE'), (399, 411, 'EMAIL'), (521, 533, 'PHONE'), (482, 497, 'PERSON'), (671, 684, 'PERSON'), (697, 735, 'DATE')]}), ('Good afternoon Mike,  As per our earlier phone conversation I generated the following work order for a request called in by Public Safety Ofc Memon:  *Work Order # 14-129321* *  Description:* 20-COOPER SQ,1 FL,LOBBY,TROUBLE ALARM ROOF SPRINKLER TANK LOW AIR,CONFIRMED WITH FM ON-CALL MIKE CALLAGHAN  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 19, 'PERSON'), (138, 147, 'PERSON'), (164, 173, 'REQUEST'), (191, 209, 'FAC'), (284, 298, 'PERSON'), (381, 394, 'PERSON'), (418, 432, 'PHONE')]}), ('Good afternoon Rose,  Thank you for contacting the Client Services Center. We submitted work order *14-128713 *as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (100, 109, 'REQUEST'), (293, 308, 'PERSON'), (332, 344, 'PHONE')]}), ('Good afternoon Victoria,  Thank you for contacting the Client Services Center. Please note that your request has been forwarded to your supervisor for approval.  We will let you know as soon as we receive a response.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(299, 312, 'PERSON'), (15, 23, 'PERSON'), (336, 350, 'PHONE')]}), ('Hello Raymond,  Please be advised of the work order below:  Description                    105-E 17TH ST,3 FL,OPEN SPACE NEAR ROOM 342C/TV,REPLACE FLICKERING LIGHT BULB.  Requester                      JOHNNY  Phone                             82726  Thank you in advance for all of your help.  Regards, Rachel Client Services Center', {'entities': [(6, 13, 'PERSON'), (91, 109, 'FAC'), (202, 208, 'PERSON'), (304, 310, 'PERSON')]}), ('Good morning AnnMarie,  Thank you for contacting the Client Services Center. We submitted work order #*14-129180*** as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(12, 21, 'PERSON'), (103, 112, 'REQUEST'), (299, 314, 'PERSON'), (376, 394, 'EMAIL')]}), ('Good afternoon Public Safety,  Please be advised of the following work order that has been entered:  Work Order:*#14-129111* Description:DIBNER LIBRARY,1 FL,AUDITORIUM,SET UP SPACE FOR EVENT ON JULY 1TH FROM 6:30PM - 7:30PM . 150 STUDENT EXPECTED, OPEN DOOR  If you have any questions or concerns, please call us.  Best Wishes, Eric H. Emma Client Services Center (212) 998-1001', {'entities': [(114, 123, 'REQUEST'), (137, 156, 'FAC'), (328, 340, 'PERSON'), (364, 378, 'PHONE'), (194, 223, 'DATE')]}), ('Good evening Anne,  Thank you for contacting the Client Services Center. We edited the work order as per your request and it has been re-sent to the building for processing. Please continue to use this number if you have any questions in the future regarding this matter.  Please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(13, 17, 'PERSON'), (339, 354, 'PERSON'), (377, 390, 'PHONE')]}), ('Good afternoon Matthew,  Thank you for contacting the Client Services Center. Please accept our apologies for this delayed response. I have forwarded your message to the appropriate personnel for advisement. We will contact you as soon as we receive a response.  We have generated work order*14-129202* to track your concern. Please refer to this number with any questions. Thank you.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 22, 'PERSON'), (292, 301, 'REQUEST'), (401, 412, 'PERSON'), (440, 452, 'PHONE')]}), ('Good Morning Sabeen,  WORK ORDER # 14-128809 has been canceled per your request.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(35, 44, 'REQUEST'), (142, 149, 'PERSON'), (13, 19, 'PERSON'), (175, 187, 'PHONE')]}), ("Good afternoon John,  We've received a follow up for the below request. Please let us know if there are any updates to provide the client. * **14-112545** **MAIN,1 FL,HALLWAY FOR RM 151,ELECTRIC SHOP TO UPGRADE CORRIDOR LIGHTING TO LED, FIXTURES ARE COOPER METALUX 2X2 (QTY:5) AND ARE AT ALUMNI HALL BASEMENT IN CBS AREA. MUST CONTACT CLIENT 48 HRS IN ADVANCE OF WORK, WIILL MEET ONSITE TO DISCUSS PLACEMENT** **Contact    OPS-DAVID M SEAWARD** **Contact Phone    2129927702** **Contact Email    david.seaward@nyu.edu*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 19, 'PERSON'), (143, 152, 'REQUEST'), (157, 166, 'FAC'), (427, 442, 'PERSON'), (464, 474, 'PHONE'), (496, 517, 'EMAIL'), (588, 603, 'PERSON'), (627, 639, 'PHONE')]}), ('Good evening Christopher,  Our office received an update from the Facility Manager approving the request to have the table/desk moved to your office. Please use work order *#14-135965* should you need to track this matter. The door issue still needs to be evaluated and the FM will contact our office once he has determined the next course of action. * **Description- JACOBS ADMIN,5 FL,RM 355A,MOVE TABLE OR SMALL CART FROM SURPLUS RM TO OFFICE SO FACULTY CAN USE IN SPACE*  In the meantime, please contact our office should you have any questions or if you require additional assistance.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(13, 24, 'PERSON'), (174, 183, 'REQUEST'), (368, 393, 'FAC'), (597, 612, 'PERSON'), (636, 648, 'PHONE')]}), ('Good afternoon James,  Thank you for contacting our office with this approval. We have generated Work Order *#**14-135965* for the move from the surplus area to the office and it has been sent to the CBS staff.  *Description- *JACOBS ADMIN,5 FL,RM 355A,MOVE TABLE OR SMALL CART FROM SURPLUS RM TO OFFICE SO FACULTY CAN USE IN SPACE  Please let us know once you have investigated the door issue and if you require any additional assistance with regards to that request.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(112, 121, 'REQUEST'), (535, 547, 'PHONE'), (15, 20, 'PERSON'), (227, 252, 'FAC'), (496, 511, 'PERSON')]}), ('Good afternoon Jim,  I hope you are well! Our office received a call today from a faculty member within Jacobs Admin regarding a few requests for his office. We already generated a work order for a trash bin to be delivered, but were wondering if there was a way to approve the moving of a small cart or table from the surplus room that Christopher Snyder can use as a printer table. He also wondered if an inner door could be installed, as he is missing on within this location. Once we receive your response, we will contact the client as needed and make the necessary arrangements.   In the meantime, please contact our office should you have any questions or if you require additional assistance.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 18, 'PERSON'), (337, 355, 'PERSON'), (709, 724, 'PERSON'), (747, 760, 'PHONE')]}), ('Hello Christopher,  Thank you for contacting the Client Services Center. Please be advised that I have forward your requests to the CBS foreman for advisement. We will let you know as soon as we receive a response. In the mean time, please feel free to contact us with any questions or concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 17, 'PERSON'), (304, 318, 'PERSON'), (342, 354, 'PHONE')]}), ('Good afternoon Lisa,  Thank you for providing us with this information. We have updated work order *14-130458* and notified Public Safety.  Please let us know if you have any questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(14, 19, 'PERSON'), (100, 109, 'REQUEST'), (253, 265, 'PHONE')]}), ('Hello Bilal,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 11, 'PERSON'), (598, 605, 'PERSON'), (631, 643, 'PHONE')]}), ('Hello Bill,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 10, 'PERSON'), (597, 604, 'PERSON'), (630, 642, 'PHONE')]}), ('Hello Mike,   Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 10, 'PERSON'), (598, 605, 'PERSON'), (631, 643, 'PHONE')]}), ('Hello Bilal,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 11, 'PERSON'), (598, 605, 'PERSON'), (631, 643, 'PHONE')]}), ('**Hello Victor,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 14, 'PERSON'), (601, 608, 'PERSON'), (634, 646, 'PHONE')]}), ('**Hello Rich,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 12, 'PERSON'), (632, 644, 'PHONE'), (599, 606, 'PERSON')]}), ('**Hello Mike,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 12, 'PERSON'), (599, 606, 'PERSON'), (632, 644, 'PHONE')]}), ('**Hello Alex,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 12, 'PERSON'), (599, 606, 'PERSON'), (632, 644, 'PHONE')]}), ('Hello Peter,   Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 11, 'PERSON'), (599, 606, 'PERSON'), (632, 644, 'PHONE')]}), ('**Hello David,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 13, 'PERSON'), (600, 607, 'PERSON'), (633, 645, 'PHONE')]}), ('Hello Peter,   Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 11, 'PERSON'), (599, 606, 'PERSON'), (632, 644, 'PHONE')]}), ('**  Hello David,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(10, 15, 'PERSON'), (602, 609, 'PERSON'), (635, 647, 'PHONE')]}), ('**Hello Bill,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(8, 12, 'PERSON'), (599, 606, 'PERSON'), (632, 644, 'PHONE')]}), ('**Hello Mike,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(632, 644, 'PHONE'), (8, 12, 'PERSON'), (599, 607, 'PERSON')]}), ('Hello Rich,   Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(598, 605, 'PERSON'), (631, 643, 'PHONE'), (6, 10, 'PERSON')]}), ('Hello Ken,  Please see the attached Excel spreadsheet which lists all open Unifirst work orders for your respective block.  Please review the file and confirm whether these requests have been completed (indicate Yes or No in Column titled "Was Request Completed") and supply any additional comments you may have for each request (the last column).  Once the updated file is returned, we will update all work orders with your notes, close out the ones that have been marked completed, and send an open work order report to Unifirst to confirm the status of any incomplete request.  Kind Regards,  Monique  Client Services Center  212-998-1001', {'entities': [(6, 9, 'PERSON'), (629, 641, 'PHONE'), (596, 603, 'PERSON')]}), ('Good afternoon Matt and Nick,  Please see the below request and let us know if there are any updates to provide the client.  *14-122867** * *2-METROTECH,8 FL,RM 874,CUT (1) COPY OF KEY, CYLINDER NUMBER AB27, CHARTFIELD 63270-11-63252, INFORM CLIENT IF KEY CAN BE SENT VIA INTER-OFFICE ENVELOPE** **Contact    ERIC MAIELLO** **Contact Email eric.maiello@nyu.edu*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tRe: Confirmation Re: Keys for MAGNET 2 MTC Room 874 Date: \tFri, 27 Jun 2014 12:46:06 -0400', {'entities': [(15, 19, 'PERSON'), (24, 28, 'PERSON'), (126, 135, 'REQUEST'), (141, 164, 'FAC'), (309, 321, 'PERSON'), (340, 360, 'EMAIL'), (431, 446, 'PERSON'), (470, 482, 'PHONE'), (623, 661, 'DATE')]}), ('Good Afternoon Eric,  Thank you for contacting the Client Services Center.  We apologize for the delay in service and any inconvenience it may have caused.  Work order number *14-122867 *has been updated for this request and has been submitted to the locksmiths to complete.  We have contacted them via email to expedite.  Please reference this number if follow up is required.  Please contact us if work is not completed or there are any questions or concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (176, 185, 'REQUEST'), (469, 484, 'PERSON'), (508, 520, 'PHONE')]}), ('Good evening Eric,  We apologize for the oversight as this work order did not need facility manager approval.  Please note that the following work order has been generated and sent to the Locksmiths for completion, as indicated by your request below.  *Work Order # 14-122867* *  Description:* 2-METROTECH,8 FL,RM 874,CUT (1) COPY OF KEY, CYLINDER NUMBER AB27, CHARTFIELD 63270-11-63252, INFORM CLIENT IF KEY CAN BE SENT VIA INTER-OFFICE ENVELOPE  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 17, 'PERSON'), (266, 275, 'REQUEST'), (294, 317, 'FAC'), (529, 542, 'PERSON'), (566, 580, 'PHONE')]}), ('Good Afternoon Scott,  Please direct this inquiry to Campus Cable at 212.998.2671 or enter a service request using their online form, found here:: HTTP://WWW.NYU.EDU/TV.MEDIA/CAMPUS.CABLE/SERVICE.REQUEST.FORM/INDEX.HTML  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (69, 81, 'PHONE'), (281, 288, 'PERSON'), (314, 326, 'PHONE')]}), ('Hey Jude,  Please pardon the inconvenience this may have caused.  WORK ORDER # 14-122811 has been submitted to the HVAC Shop for completion.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(4, 8, 'PERSON'), (79, 88, 'REQUEST'), (235, 247, 'PHONE'), (202, 209, 'PERSON')]}), ('Good Afternoon Jude,  Thank you for contacting the Client Services Center.  WORK ORDER # 14-122811  has been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 19, 'PERSON'), (89, 98, 'REQUEST'), (223, 235, 'PHONE'), (250, 268, 'EMAIL'), (323, 330, 'PERSON'), (356, 368, 'PHONE')]}), ('Good evening Richard,  Thanks for following up with us and we apologize for the oversight.  The work order has been updated to reflect the correct date 7/15.  My coworker made a typo in her response.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 20, 'PERSON'), (152, 156, 'DATE'), (282, 295, 'PERSON'), (319, 333, 'PHONE')]}), ('Good afternoon Victor,  Thank you for contacting our office with this update. We have edited the Work Order to note that AKA will be on site on July 17th. * ***Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 21, 'PERSON'), (144, 153, 'DATE'), (274, 289, 'PERSON'), (313, 325, 'PHONE')]}), ('Good evening Mike.  Please be advised the client followed up with us in regards to this key request from 6/11, hoping for an expedited response.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 17, 'PERSON'), (105, 109, 'DATE'), (227, 240, 'PERSON'), (264, 278, 'PHONE')]}), ('Good afternoon All,  The following requests has not been completed as per the client. She has requested an update on the status of these repairs.  /14-125926// //SCHWARTZ HALL,9 FL,RM 901S,ENTRY DOOR ? GETS STUCK BECAUSE THE LATCH DOES NOT GO ALL THE WAY INTO THE FACE PLATE WHEN OPENING.// / / 14-125928 SCHWARTZ HALL,9 FL,RM 902S,ENTRY DOOR NOT LOCKING BECAUSE THE LATCH DOES NOT GO ALL THE WAY INTO THE STRIKE PLATE.  14-125929 SCHWARTZ HALL,9 FL,RM 902S,CLOSET DOOR GETS STUCK BECAUSE HINGES ARE MISALIGNED.  Contact    CYNTHIA CHONG Contact Phone    (212) 998-9556 Contact Email    cynthia.chong@nyu.edu/  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tRE: CONFIRMATION - Re: Problems with locks on several doors Date: \tFri, 27 Jun 2014 09:52:51 -0400', {'entities': [(148, 157, 'REQUEST'), (162, 188, 'FAC'), (295, 304, 'REQUEST'), (305, 331, 'FAC'), (421, 430, 'REQUEST'), (431, 457, 'FAC'), (524, 537, 'PERSON'), (555, 569, 'PHONE'), (587, 608, 'PERSON'), (679, 694, 'PERSON'), (718, 730, 'PHONE'), (886, 917, 'DATE')]}), ('Good afternoon Johnathan,  Thank you for contacting the Client Services Center. We submitted work order #:**14-128896 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 24, 'PERSON'), (108, 117, 'REQUEST'), (316, 331, 'PERSON'), (355, 367, 'PHONE')]}), ('Hello Jennifer,  Thank you for contacting the Client Services Center. Please be advised that I have forward your requests to the facility manager for advisement. We will let you know as soon as we receive a response.  In the mean time, please feel free to contact us with any questions or concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 14, 'PERSON'), (345, 357, 'PHONE'), (307, 321, 'PERSON')]}), ('Good afternoon Jennifer,  Thank you for contacting the Client Services Center. We submitted work order *14-130501 *as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 23, 'PERSON'), (104, 113, 'REQUEST'), (336, 348, 'PHONE'), (297, 312, 'PERSON')]}), ('Good evening Ian,  Thanks for following up with us, and we apologize for any inconvenience.  The Carpenters have been fully booked this summer which may partially explain the delay.  I have updated the work order *14-127465* with your notes and resent to the Carpenters, asking them to expedite.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 16, 'PERSON'), (214, 223, 'REQUEST'), (415, 429, 'PHONE'), (378, 391, 'PERSON')]}), ('Good afternoon Ian,  Thank you for contacting the Client Services Center. We submitted work order *14-127465 *as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 18, 'PERSON'), (99, 108, 'REQUEST'), (292, 307, 'PERSON'), (331, 344, 'PHONE')]}), ('Good afternoon Celia,  I apologize for the delay in service regarding the shade and I have forwarded your request to the facility manager and marked it as urgent, which hopefully will expedite the process.  Please feel free to contact us if you have any questions or concerns.  Best Wishes, Eric H. Emma Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (291, 303, 'PERSON'), (327, 341, 'PHONE')]}), ('Hello Celia,  Thank you for contacting the Client Services Center. Please be advised that I have forward your requests to the facility manager for advisement. We will let you know as soon as we receive a response.  In the mean time, please feel free to contact us with any questions or concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 11, 'PERSON'), (304, 318, 'PERSON'), (342, 354, 'PHONE')]}), ('Good Afternoon Celia,  Thank you for contacting the Client Services Center.  *WORK ORDER #  14-117867*  has been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(254, 272, 'EMAIL'), (92, 101, 'REQUEST'), (15, 20, 'PERSON'), (227, 239, 'PHONE'), (327, 334, 'PERSON'), (360, 372, 'PHONE')]}), ('Good afternoon Michelle,  Thank you for this update. I will update the records with the below information and have accounts payable run the chartfield in a few days.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 23, 'PERSON'), (173, 188, 'PERSON'), (212, 224, 'PHONE')]}), ('Good afternoon Michelle,  Unfortunately, we were still unable to process payment to chartfield /63130-20-74200-M7208/ for the 3 moving services requests below. Please provide an updated chartfield as soon as possible as charges need to be made before the end of the fiscal year.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 23, 'PERSON'), (348, 363, 'PERSON'), (387, 399, 'PHONE')]}), ('Good evening Michelle,  We received an "invalid chartfield" error when we entered the chartfield for payment. Please provide a valid chartfield for the work orders below.  *14-026101** **WARREN WVR,6 FL,RM 624,DEPENDABLE TO MOVE 10-15 BOXES TO KIMMEL CENTER, 4 FL, E&L DRESSING ROOM AT 4:30PM ON 10/31, SEE ELAINE FLUDGATE IN 605,CONFIRMED #4755** **63130-20-74200-M7208** ** **14-026108** **WARREN WVR,6 FL,RM 624,DEPENDABLE TO MOVE ANY REMAINING BOXES TO KIMMEL CENTER, 4 FL, E&L DRESSING ROOM AT 9:00AM ON 11/01, SEE ELAINE FLUDGATE IN 605,CONFIRMED #4756 * CHART FIELD 63130-20-74200-M7208** ** **14-027285** **KIMMEL CENTER,6 FL,RM 605,DEPENDABLE TO MOVE ALL THE BOXES MARKED MICHELLE SHIN, MATHEMATICS DEPARTMENT FROM KIMMEL 605 AND DELIVER THEM TO 251 MERCER STREET 624 AT 9:30AM ON MONDAY, NOVEMBER 11TH. ** **63130-20-74200-M7208** **DEPENDABLE REF #5519** * Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(173, 182, 'REQUEST'), (187, 209, 'FAC'), (378, 387, 'REQUEST'), (392, 414, 'FAC'), (457, 476, 'FAC'), (244, 263, 'FAC'), (499, 514, 'DATE'), (286, 301, 'DATE'), (615, 640, 'DATE'), (681, 694, 'PERSON'), (520, 535, 'PERSON'), (601, 610, 'REQUEST'), (724, 734, 'FAC'), (755, 776, 'FAC'), (780, 811, 'DATE'), (936, 951, 'PERSON'), (975, 987, 'PHONE')]}), ('Good afternoon Michelle,  Thank you for contacting the Client Service Center. Please be advised work orders below:  _Work Order_ 14-027285 _Description_    KIMMEL CENTER,6 FL,RM 605,DEPENDABLE TO MOVE ALL THE BOXES MARKED MICHELLE SHIN, MATHEMATICS DEPARTMENT FROM KIMMEL 605 AND DELIVER THEM TO 251 MERCER STREET 624 AT 9:30AM ON MONDAY, NOVEMBER 11TH. 63130-20-74200-M7208 DEPENDABLE REF #5519  _Work Order _ 14-026108 _Description_    WARREN WVR,6 FL,RM 624,DEPENDABLE TO MOVE ANY REMAINING BOXES TO KIMMEL CENTER, 4 FL, E&L DRESSING ROOM AT 9:00AM ON 11/01, SEE ELAINE FLUDGATE IN 605,CONFIRMED #4756 * CHART FIELD 63130-20-74200-M7208  Please see both attachments which have referenced these work orders.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(15, 23, 'PERSON'), (129, 138, 'REQUEST'), (156, 181, 'FAC'), (222, 235, 'PERSON'), (265, 275, 'FAC'), (296, 317, 'FAC'), (321, 352, 'DATE'), (411, 420, 'REQUEST'), (438, 460, 'FAC'), (503, 522, 'FAC'), (545, 560, 'DATE'), (566, 581, 'PERSON'), (757, 769, 'PHONE'), (718, 733, 'PERSON')]}), ('Good afternoon Ed,  Thank you for contacting our office with this update. We have added this information to the work order and it has been updated. Please let us know if you require any further assistance or if you need us to generate a request with regards to access for any contractor repair work.  Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 17, 'PERSON'), (415, 430, 'PERSON'), (454, 466, 'PHONE')]}), ('Good afternoon all,  Please be advised of the below work order submitted by the facility manager David Pinero:  *Work Order # 14-127880* *  Description:* KIMMEL CENTER,1 FL,SKIRBALL BALCONY/LOBBY DOORS,INVESTIGATE AND REPAIR LOOSE WIRE FOR NEWLY INSTALLED CARD READER  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(154, 182, 'FAC'), (350, 363, 'PERSON'), (97, 109, 'PERSON'), (126, 135, 'REQUEST'), (387, 401, 'PHONE')]}), ('Good afternoon Kevin,  Thanks for following up with us and we apologize the work was not totally completed as requested.  I have updated the work order *14-106419* with your notes and forwarded on to the Utility shop manager for advisement.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (153, 162, 'REQUEST'), (323, 336, 'PERSON'), (360, 374, 'PHONE')]}), ('Good afternoon Susanna,  Thank you for contacting our office with regards to this request. We have forwarded your message to the Super and to the Facility Manager for assistance. Should you have any additional questions or comments regarding this matter, please contact our office and we will assist in any way we can.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 22, 'PERSON'), (327, 342, 'PERSON'), (366, 378, 'PHONE')]}), ("Good afternoon Erik,  I hope you are well! Our office received the below message from Susanna Stein with regards to access to an apartment within 1-Wash Sq V (Apt 15A). Please advise if there are any additional steps our office should take to confirm this matter, as the sub-tenant was CC'd in the original message and also, if a work order will be required to for this request.  Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001   -------- Original Message -------- Subject: \tAccess to 1 WSV #15A Date: \tThu, 26 Jun 2014 11:05:54 -0400", {'entities': [(15, 19, 'PERSON'), (86, 99, 'PERSON'), (146, 167, 'FAC'), (494, 509, 'PERSON'), (533, 545, 'PHONE'), (603, 613, 'FAC'), (621, 652, 'DATE')]}), ("Good afternoon Kristen,  Thank you for following up with our office. We've forwarded your email to the maintenance staff for their advisement. We will contact you as soon as we receive a response.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 22, 'PERSON'), (266, 281, 'PERSON'), (305, 317, 'PHONE')]}), ('Good afternoon All,  Please see the below follow up message we received from the tenant and let us know if a new work order should be generated.  *14-119457** * *1-WASH SQ V,16 FL,APT 16I,BEDROOM AC UNIT FRONT FACE FALLING OFF, UNIT IS VERY OLD** **Contact    KRISTEN SOSULSKI** **Contact Phone    212.998.0994** **Contact Email    ks123@nyu.edu*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tRe: Confirmation Re: Bedroom Air conditioner Date: \tWed, 16 Jul 2014 14:05:18 -0400', {'entities': [(147, 156, 'REQUEST'), (162, 187, 'FAC'), (260, 276, 'PERSON'), (298, 310, 'PHONE'), (332, 345, 'EMAIL'), (416, 431, 'PERSON'), (455, 467, 'PHONE'), (608, 639, 'DATE')]}), ('Good afternoon Kristen,  Thank you for contacting the Client Services Center. Please note that the following work order has been generated and sent to the building staff for completion, as indicated by your request below.  *Work Order: **14-119457* *  Description:* 1-WASH SQ V,16 FL,APT 16I,BEDROOM AC UNIT FRONT FACE FALLING OFF, UNIT IS VERY OLD  _Please advise whether you provide Permission To Enter (PTE) or if you would prefer to admit the building service personnel (TWA)._  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 22, 'PERSON'), (238, 247, 'REQUEST'), (266, 291, 'FAC'), (564, 577, 'PERSON'), (601, 615, 'PHONE')]}), ('Hello David and Mitchell,  A client Janet Bunde in the Library Archives followed up with us again this afternoon, noting that the toilet was again clogged. I created a new work order *14-131483* and confirmed with Edny.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(6, 11, 'PERSON'), (184, 193, 'REQUEST'), (339, 353, 'PHONE'), (16, 24, 'PERSON'), (36, 47, 'PERSON'), (214, 218, 'PERSON'), (302, 315, 'PERSON')]}), ('Good Afternoon Mitchell,  Thank you Mitchell.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(140, 152, 'PHONE'), (36, 44, 'PERSON'), (107, 114, 'PERSON'), (15, 23, 'PERSON')]}), ('Good afternoon David,  Please be advised I received a call from Nancy Crrico x82644 in regards to persistent problems with a clogged toilet in the Eastside Womens restroom.  She explained that there have been several calls in the past three months, and that repairs only seemed to last a couple of days.  She would like a more permanent repair.  I found five workers generated in that span of time and attached in spreadsheet.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (546, 560, 'PHONE'), (64, 76, 'PERSON'), (509, 522, 'PERSON')]}), ('Good Afternoon Mark,  Thanks for clarifying.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 19, 'PERSON'), (139, 151, 'PHONE'), (106, 113, 'PERSON')]}), ('Good afternoon Mark,  Thank you for contacting the Client Services Center. We submitted work order *#14-131418 *as per your request. Please refer to this number if you have any questions in the future regarding this matter. /Please note if the location for this work order is correct, as MacDougal Alley is outside and this is a faculty housing location.  /*Description- *21-MACDOUGAL,1 FL,OUTSIDE,STAFF TO ASSIST TOR CONTRACTING AS NEEDED WHILE WORK TALES PLACE ON MACDOUGAL ALLEY CONCRETE AND ASPHALT REPAIRS TODAY, 7/3 THROUGH WEDNESDAY. TO OCCUR FROM 8AM TO 4PM, AND NEXT WEEK 7/7 - 7/9.  Please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (518, 539, 'DATE'), (555, 558, 'DATE'), (562, 565, 'DATE'), (581, 584, 'DATE'), (101, 110, 'REQUEST'), (288, 303, 'FAC'), (372, 390, 'FAC'), (466, 481, 'FAC'), (587, 590, 'DATE'), (659, 674, 'PERSON'), (698, 710, 'PHONE')]}), ('Good Afternoon Franklin,  This has been added to our records.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 23, 'PERSON'), (123, 130, 'PERSON'), (156, 168, 'PHONE')]}), ('Good afternoon Stevin,  Thanks for following up with us and we apologize for the long delay.  I have updated the work order *14-095465* and forwarded on the Electric shop manager, asking for an urgent response.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 21, 'PERSON'), (125, 134, 'REQUEST'), (293, 306, 'PERSON'), (330, 344, 'PHONE')]}), ('Good afternoon Stevin,  Thank you for contacting the Client Services Center. We submitted work order #*14-095465* to the Electric Shop, as per your request. Please refer to this number if you have any questions in the future regarding this matter.  *Description- *721-BROADWAY,10 FL,RM 1045,1046,1047,1048,1049 &1050,PROVIDE ESTIMATE TO REPLACE DIMMERS IN LISTED ROOMS  Please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 21, 'PERSON'), (103, 112, 'REQUEST'), (264, 290, 'FAC'), (436, 451, 'PERSON')]}), ('Good afternoon David,  Thank you for contacting the Client Services Center. We submitted work order *14-130497* to building staff, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (101, 110, 'REQUEST'), (325, 336, 'PERSON'), (364, 376, 'PHONE')]}), ('Good evening Jim,  Thank you for contacting the Client Services Center. We submitted work order #*14-130499* as per your request. Please refer to this number if you have any questions in the future regarding this matter.  *Description- *JACOBS ACADEMIC,8 FL,ELECTRIC CLOSET,REINSTALL ELECTRIC PANEL COVER. COVER IS IN ROOM.  Please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(13, 16, 'PERSON'), (98, 107, 'REQUEST'), (237, 257, 'FAC'), (391, 406, 'PERSON'), (430, 442, 'PHONE')]}), ('Good afternoon Franklin,  We submitted work order *14-130278* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 23, 'PERSON'), (51, 60, 'REQUEST'), (283, 295, 'PHONE'), (244, 259, 'PERSON')]}), ('Good afternoon Kevin,  Thank you for contacting the Client Services Center. We submitted work order *14-130293* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 20, 'PERSON'), (101, 110, 'REQUEST'), (294, 309, 'PERSON'), (333, 345, 'PHONE')]}), ("Good afternoon CBS,  Please note, the below work order has not been completed as per the client  *14-128299** * *WEISSMAN HALL,3 FL,3W,VACUUM ENTIRE RECEPTION AREA AND UNDER DESKS. STEAM CLEAN STAIN NEXT TO WATER COOLER OUTSIDE OF DOCTOR CHO'S OFFICE RM 314.** **Contact    MARGIE PEREZ** **Contact Phone    2129989681** **Contact Email    mp92@nyu.edu*  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tRe: Work Request Number: 14-128299 Date: \tTue, 1 Jul 2014 09:50:56 -0400", {'entities': [(98, 107, 'REQUEST'), (113, 131, 'FAC'), (274, 286, 'PERSON'), (308, 318, 'PHONE'), (340, 352, 'EMAIL'), (423, 438, 'PERSON'), (462, 474, 'PHONE'), (605, 635, 'FAC'), (588, 597, 'REQUEST')]}), ('Hello Zack,  Sorry about the delay.  Regards, Rachel Client Services Center', {'entities': [(6, 10, 'PERSON'), (46, 52, 'PERSON')]}), ('Good afternoon Kristina,  To make things easier and hopefully expedite things for you, I have attached a copy of the work order to this e-mail.   Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 23, 'PERSON'), (231, 246, 'PERSON'), (270, 282, 'PHONE')]}), ('Good morning Kristina,  Thank you for contacting the Client Services Center.  We submitted work order #: *14-096300/002 *as per your request.  Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  Alexis Snowden Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(106, 115, 'REQUEST'), (13, 21, 'PERSON'), (304, 318, 'PERSON'), (342, 354, 'PHONE')]}), ('Good Afternoon Judy,  This request has been forwarded to the facility manager for review and advisement.  Once a response has been received, you will be notified.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 19, 'PERSON'), (224, 231, 'PERSON'), (257, 269, 'PHONE')]}), ('Good Afternoon Matt,  Please specify which building you are requesting to have the cores changed.  Once this information is provided, a work order will be generated.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(260, 272, 'PHONE'), (15, 19, 'PERSON'), (227, 234, 'PERSON')]}), ('Good Afternoon Daniel,  Kami will be notified to meet with Jose on Monday July 7 at 8am in 1-B-WASH PL (Route L223).  Aurelio and Mike,  Please advise if Kami will be available for this resizing.  Daniel,  Please indicate if Jose will meet Kami in front of his locker on Monday 7/7 at 8AM.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 21, 'PERSON'), (24, 28, 'PERSON'), (59, 63, 'PERSON'), (67, 87, 'DATE'), (91, 102, 'FAC'), (118, 125, 'PERSON'), (130, 134, 'PERSON'), (154, 158, 'PERSON'), (271, 288, 'DATE'), (197, 203, 'PERSON'), (225, 229, 'PERSON'), (240, 244, 'PERSON'), (351, 358, 'PERSON')]}), ("Hello Daniel,  What is the longest inseam you currently stock?  With regards to Kami, please have your routeman measure him since Kami only has only _*one*_ pair of pants and there have been at least three (3) replacements thus far that have each been incorrect.  As a result, please advise when the routeman will be available to obtain Kami's measurements even though an order was already placed.  Please advise when the routeman will measure Kami to ensure the new pants are the correct size.  Please also advise if the routeman can bring a temp uniform to ensure a proper fit.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001", {'entities': [(6, 12, 'PERSON'), (130, 134, 'PERSON'), (444, 448, 'PERSON'), (337, 341, 'PERSON'), (641, 648, 'PERSON'), (674, 686, 'PHONE')]}), ('Good afternoon Paulo,  I apologize for any confusion, but your request was accidentally processed twice. Please refer to work order *14-130363* with any questions regarding this matter. Thank you.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (133, 142, 'REQUEST'), (213, 224, 'PERSON'), (252, 264, 'PHONE')]}), ('Good afternoon Paulo,  Thank you for contacting the Client Services Center. We submitted work order *14-130364* to Card Services, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (101, 110, 'REQUEST'), (324, 335, 'PERSON'), (363, 375, 'PHONE')]}), ('Good afternoon All,  Please see the below email.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tID Tech Camp for Tuesday, July 1 Date: \tMon, 30 Jun 2014 17:02:39 -0400', {'entities': [(56, 71, 'PERSON'), (95, 107, 'PHONE'), (222, 228, 'DATE'), (236, 267, 'DATE')]}), ('Good afternoon Raymond,  We submitted work order *14-129087***as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (50, 59, 'REQUEST'), (244, 259, 'PERSON'), (283, 296, 'PHONE')]}), ('Good morning Andrew,  Thank you for contacting the Client Services Center. Please note work order/14-129366/ has been labeled as completed and states the following: _ Description_: MEYER,8 FL,ROOM 828 (WITHIN ROOM 822),REPLACE FLUORESCENT BULBS IN HOODS 3 AND 4.  _SHOP_COMPLETED_: 06/30/2014 | S.GARCIA W/O TO BE COMPLETED BY BLDG OPS.  Please note work order _*14-129452*_ was generated for the building operators to investigate. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(13, 19, 'PERSON'), (98, 107, 'REQUEST'), (181, 200, 'FAC'), (282, 292, 'DATE'), (363, 372, 'REQUEST'), (295, 303, 'PERSON'), (593, 608, 'PERSON'), (632, 644, 'PHONE')]}), ('Good afternoon Catherine,  The facility manager will be able to look into an outside paint vendor to complete this request.  Michael, please advise as to whether facilities can cover these charges and if an outside vendor can complete this work.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 24, 'PERSON'), (125, 132, 'PERSON'), (315, 330, 'PERSON'), (354, 366, 'PHONE')]}), ("Good morning Jessica,  I hope this message finds you well. Please note that the following work order has been generated for room 1603 in Gramercy Green, reporting a severe leak and water damage.  I reached out to the Engineers by radio to see who responded to the request, however I was unable to speak with anyone. I called Public Safety and was informed that this request was reported to Sgt. Wright and the overnight Engineer, Nick.     *Work Order* _#14-130077 _    *Description*: GRAMERCY GREEN,16 FL,ROOM: 1603,CEILING LEAKING -    TOILET IN FLOOR ABOVE WAS LEAKING AND CLOGGED. WATER DAMAGE TO:    ENTRY WAY AND WALL ABOVE REFRIGERATOR. CEILING\\WALLS BUBBLING AND    FALLING OFF.CLIENT CONFIRMED WITH P.S. AND ENGINEER.SEE EXTRA    DESCRIPTION.  *Extra Description*: I AWOKE TO THE SOUND OF POURING WATER AT 3AM, REALIZED THAT THE CEILING WAS LEAKING IN MULTIPLE PLACES (AT LEAST 4 SEPARATE LOCATIONS), AND WENT DOWNSTAIRS TO ALERT THE GUARD AND FIGURE OUT WHAT WAS GOING ON. AN ENGINEER CAME TO CHECK IT OUT AND THEN REALIZED THAT THE ROOM ABOVE US HAD A CLOGGED AND FLOODING TOILET. THE FLOOR IS FLOODED AND PROBABLY GALLONS OF WATER HAVE COME DOWN (I EMPTIED A LARGE POT MULTIPLE TIMES). THERE IS MAJOR WATER DAMAGE IN THE ENTRY WAY AND OVER THE REFRIGERATOR AND SOME OF THE PAINT ON THE CEILING AND WALLS HAVE BUBBLED UP AND/OR FALLEN OFF.  If needed, here is the client's contact information.     *Name*: Katherine McConnell    *Phone*: 207-650-4400    *Email*: katiemcconnell93@gmail.com  Please feel free to contact us if you have any questions or concerns.  Thank you for your time and assistance, Kerri Campbell  Client Services Center (212) 998-1001", {'entities': [(13, 20, 'PERSON'), (129, 151, 'FAC'), (455, 464, 'REQUEST'), (430, 434, 'PERSON'), (395, 401, 'PERSON'), (485, 516, 'FAC'), (1417, 1436, 'PERSON'), (1449, 1461, 'PHONE'), (1474, 1500, 'EMAIL'), (1613, 1627, 'PERSON'), (1652, 1666, 'PHONE')]}), ('Good morning,  Please note the follow work order *14-129463* has been generated as per the clients request.  Please let us know if you have any other questions/concerns.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail  -------- Original Message -------- Subject: \tGTS Building Access Request - Marina Thomatos Date: \tMon, 30 Jun 2014 09:32:51 -0400', {'entities': [(50, 59, 'REQUEST'), (178, 193, 'PERSON'), (217, 229, 'PHONE'), (348, 363, 'PERSON'), (371, 402, 'DATE')]}), ('Good afternoon Tina,  Thank you for contacting the Client Services Center. We submitted work order #*14-129463*** as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (101, 110, 'REQUEST'), (297, 312, 'PERSON'), (374, 392, 'EMAIL')]}), ('Hello Doris,  Good afternoon.  Thank you, we will close the work orders as quickly as possible.  Regards, Brent Maddox Client Services Center (212)998-1001', {'entities': [(6, 11, 'PERSON'), (106, 118, 'PERSON'), (142, 155, 'PHONE')]}), ('Good afternoon Frank,  Thank you for contacting the Client Services Center. We submitted work order #:**14-129117 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (104, 113, 'REQUEST'), (312, 327, 'PERSON'), (351, 363, 'PHONE')]}), ('Good afternoon All,  Please see attached work order *14-129582.*  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail   -------- Original Message -------- Subject: \tAccess Biology: 3 More (Chen, Feher, + Rahnan) Date: \tMon, 30 Jun 2014 11:35:41 -0400', {'entities': [(72, 87, 'PERSON'), (236, 240, 'PERSON'), (242, 247, 'PERSON'), (53, 62, 'REQUEST'), (111, 123, 'PHONE'), (251, 257, 'PERSON'), (266, 297, 'DATE')]}), ("Good afternoon Leah,  Thank you for contacting the Client Services Center. We've forwarded your email to the VP for Communications for advisement. We will contact you as soon as we receive a response.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 19, 'PERSON'), (270, 285, 'PERSON'), (309, 321, 'PHONE')]}), ('Hello Mike,  Please be advised that WO# 14-131495 was updated with the times provided below.  726-BROADWAY,1 FL,BOOKSTORE,PROVIDE GUARD COVERAGE FOR (8) NIGHTS DUE TO CONSTRUCTION. FROM 7/7 UNTIL 7/16. SEE EXTRA DESCRIPTION FOR TIMES. (64305-10-20151)  MONDAY 7/7 8PM TO 10AM  TUESDAY 7/8 8PM TO 10AM  WEDNESDAY 7/9 8PM TO 10AM  THURSDAY 7/10 8PM TO 10AM  FRIDAY 7/11 8PM TO 10AM  MONDAY 7/14 8PM TO 10AM  TUESDAY 7/15 8PM TO 10AM  WEDNESDAY 7/16 8PM TO 10AM  TOTAL (96) HOURS  Best,  Sonnett Smyrna Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(6, 10, 'PERSON'), (40, 49, 'REQUEST'), (94, 111, 'FAC'), (186, 189, 'DATE'), (196, 200, 'DATE'), (253, 267, 'DATE'), (271, 275, 'DATE'), (277, 292, 'DATE'), (296, 300, 'DATE'), (302, 319, 'DATE'), (323, 327, 'DATE'), (329, 346, 'DATE'), (350, 354, 'DATE'), (356, 371, 'DATE'), (375, 379, 'DATE'), (381, 396, 'DATE'), (400, 404, 'DATE'), (406, 422, 'DATE'), (426, 430, 'DATE'), (432, 450, 'DATE'), (454, 458, 'DATE'), (485, 499, 'PERSON'), (523, 535, 'PHONE')]}), ('Hello Mike,  The below is the information we sent you on June 25th that we received from Public Safety. Please let us know if you need any more additional information from Public Safety:  /"Please note the hourly rate for the contract guard is $17.45 per hour up to 40 hours per week.  Over 40 hours the hourly// //rate is $26.18.  Therefore, for the week of 7/7-7/11, it would be:// // //40 hours X $17.45 = $698.00 plus// //20 hours X $26.18 =   523.60// //for a total of   =        $1,221.60// // //For the week of 7/14-7/16 it would be:// // //36 hours X $17.45  =  $628.20// // //The grand total would be $1,849.80.// // //Please let me know at your earliest convenience when we can schedule the contract guard."/  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(6, 10, 'PERSON'), (57, 66, 'DATE'), (805, 820, 'PERSON'), (844, 856, 'PHONE')]}), ('Hello Mike,  We received the following response from Public Safety:  /"Please note the hourly rate for the contract guard is $17.45 per hour up to 40 hours per week.  Over 40 hours the hourly// //rate is $26.18.  Therefore, for the week of 7/7-7/11, it would be:// // //40 hours X $17.45 = $698.00 plus// //20 hours X $26.18 =   523.60// //for a total of   =        $1,221.60// // //For the week of 7/14-7/16 it would be:// // //36 hours X $17.45  =  $628.20// // //The grand total would be $1,849.80.// // //Please let me know at your earliest convenience when we can schedule the contract guard."/  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(6, 10, 'PERSON'), (608, 619, 'PERSON'), (647, 659, 'PHONE')]}), ('Good afternoon Catherine,  We have forwarded your e-mail to the facility manager as soon as he gets back to us, we will let you know.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 24, 'PERSON'), (220, 235, 'PERSON'), (259, 271, 'PHONE')]}), ('Good afternoon Carbie,  Thank you for contacting the Client Services Center. We submitted work order *14-129713* as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (295, 310, 'PERSON'), (334, 346, 'PHONE'), (102, 111, 'REQUEST')]}), ('Good afternoon David, Shanelle,  We received a follow-up from the client regarding this request. Is there a quote/any information that we may pass on to him? Thank you!  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (22, 30, 'PERSON'), (177, 188, 'PERSON'), (216, 228, 'PHONE')]}), ('Good afternoon Chyng,  Thank you for contacting the Client Services Center. Please clarify the location for your request. We can generate a work order as soon as we receive this information.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 20, 'PERSON'), (260, 275, 'PERSON'), (299, 311, 'PHONE')]}), ("Good afternoon Radhica,  We've received a follow up from the client regarding moving request #14-023429. Account 63130-25-74100-F7018 has not been charged and the department grant ends at the end of the fiscal year. Please let us know how soon you will be able to process these journals for the clients.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 22, 'PERSON'), (94, 103, 'REQUEST'), (373, 388, 'PERSON')]}), ('Good afternoon Radhica,  Please also see the updated October journals. Please let us know if these accounts are still invalid so we can follow up with the clients.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (171, 186, 'PERSON'), (210, 222, 'PHONE')]}), ('Good afternoon Radhica,  Thank your for your response. We will follow up with the clients regarding the invalid/incomplete chartfields and resubmit the updated journal for December.  Please also see the attached journals for January through April, charged against the same purchase order.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(172, 180, 'DATE'), (241, 246, 'DATE'), (15, 22, 'PERSON'), (225, 232, 'DATE'), (358, 373, 'PERSON')]}), ('Good afternoon Radhica,  I hope this email finds you well. Attached are our DTM Journals for December 2013. Please let us know if there are any chargebacks of discrepancy.  Please let us know if you have any questions.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (93, 106, 'DATE'), (226, 241, 'PERSON'), (265, 277, 'PHONE')]}), ('Good afternoon Kevin and Helen,  Thanks for following up with us and letting us know.  I have updated the work order *14-130152* with your notes.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (25, 30, 'PERSON'), (118, 127, 'REQUEST'), (228, 241, 'PERSON'), (265, 279, 'PHONE')]}), ('Good Morning,  Please approve/deny the following request:  CUSTOMER REQUEST# 37421015 - WEISSMAN HALL,6 FL,NAGLE,ON WEDNESDAY JULY 2 AT 8:30 AM - SET UP TABLES AND CHAIRS IN THE FRONT OF THE ROOM CONFERENCE STYLE TO ACCOMMODATE 29 PEOPLE.  CONTACT - CARINE JOSEPH  PHONE - 89718  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(88, 106, 'FAC'), (116, 143, 'DATE'), (340, 347, 'PERSON'), (373, 385, 'PHONE')]}), ('Good afternoon Gretchen,  Thanks for following up with us and letting us know.  I have updated the work order *14-128739* with your notes.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 23, 'PERSON'), (221, 234, 'PERSON'), (111, 120, 'REQUEST'), (258, 272, 'PHONE')]}), ('Good Afternoon Susan,  Thank you for contacting the Client Services Center.  WORK ORDER # 14-130226  has been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (90, 99, 'REQUEST'), (357, 369, 'PHONE'), (224, 236, 'PHONE'), (251, 269, 'EMAIL'), (324, 331, 'PERSON')]}), ('Good Morning,  Enclosed is WORK ORDER # 14-128802 - 44-W 4TH ST,ENTIRE BLDG,PNG ISSUED TO TIMMY SHUM N14536678 FOR KMC AND TISCH EFFECTIVE IMMEDIATELY UNTIL 6/1/2015.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(40, 49, 'REQUEST'), (52, 75, 'FAC'), (123, 128, 'FAC'), (157, 165, 'DATE'), (228, 235, 'PERSON'), (261, 273, 'PHONE')]}), ('Good afternoon Victor, Peter,  Thank you for contacting the Client Services Center. We submitted work order *14-129204* to to the Locksmith Shop, as per your request.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 21, 'PERSON'), (23, 28, 'PERSON'), (109, 118, 'REQUEST'), (248, 259, 'PERSON')]}), ('Good afternoon Eric,  Thank you for contacting the Client Services Center. We submitted work order #*14-130351 ***as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(15, 19, 'PERSON'), (101, 110, 'REQUEST'), (374, 392, 'EMAIL'), (297, 312, 'PERSON')]}), ("Good afternoon Tunazzina,  We apologize for the ongoing delays you've experienced regarding these requests. We will reach out to the Facility Manager again for an update. We will let you know as soon as we receive a response.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 24, 'PERSON'), (295, 310, 'PERSON'), (334, 346, 'PHONE')]}), ('Good morning Rohan,  Thank you for contacting the Client Services Center. We submitted work order #*14-130172 ***as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(13, 18, 'PERSON'), (100, 109, 'REQUEST'), (296, 311, 'PERSON'), (373, 391, 'EMAIL')]}), ('Good afternoon Alyssa,  We submitted work order*14-129497* for the building super to allow access. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(15, 21, 'PERSON'), (260, 275, 'PERSON'), (48, 57, 'REQUEST'), (299, 311, 'PHONE')]}), ('Good evening Rose,  Thank you for contacting the Client Services Center. We submitted work order *14-115119 *to the Carpentry Shop to provide an estimate, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(13, 17, 'PERSON'), (98, 107, 'REQUEST'), (349, 360, 'PERSON'), (388, 400, 'PHONE')]}), ('Good Morning,  Please be advised Helen Wong is requesting to cancel the guards scheduled to work at Alumni today (see WORK ORDER # 14-113321 below).  WORK ORDER # 14-113321 - ALUMNI HALL,1 FL,PROVIDE 2 FULL TIME GUARDS FOR LL11 PROJECT STARTING 5/28 TO 8/15 UNLESS NOTIFIED, FROM 8:30AM TO 630PM DAILY, GUARDS WILL BE POSTED ON 7 FL AND 13 FL TO PREVENT ACCESS TO ROOFS, EXCEPT FOR CONTRACTOR 80010-52-20220-WSQPG-D13B4  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001   -------- Original Message -------- Subject: \tGuards - Alumni Hall Date: \tTue, 1 Jul 2014 09:01:40 -0400', {'entities': [(33, 43, 'PERSON'), (131, 140, 'REQUEST'), (163, 172, 'REQUEST'), (175, 191, 'FAC'), (245, 249, 'DATE'), (253, 257, 'DATE'), (280, 286, 'DATE'), (290, 295, 'DATE'), (481, 488, 'PERSON'), (514, 526, 'PHONE'), (602, 632, 'DATE')]}), ('Good afternoon Mike,  Please be advised of the below work order forwarded on to the HVAC shop:  *Work Order # 14-129819* *  Description:* 721-BROADWAY,7 FL,RMS 706,709 AND 711,TOO COLD. CONFIRMED WITH ROBERT. *AS PER OPS ROBERT SEND TO HVAC, SEE EXTRA DESCRIPTION  RM 709 IS 70 DEGREES OVERALL TEMP IS 71.2, RM 711 DISCHARGE IS 66 OVERALL TEMP IS 70, RM 706 DISCHARGE IS 71 OVERALL 72.1, NO UNITS RUNNING IN 706, BUT THERE IS AIR IN 709 AND 711 NO CLIENTS PRESENT IN 3 ROOMS  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 19, 'PERSON'), (110, 119, 'REQUEST'), (138, 163, 'FAC'), (201, 207, 'PERSON'), (221, 227, 'PERSON'), (557, 570, 'PERSON')]}), ('Good afternoon Tony,  Thank you for contacting the Client Services Center. We submitted work order *14-129332* to building staff, as per your request. You may refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions or concerns.  Best regards,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(15, 19, 'PERSON'), (100, 109, 'REQUEST'), (324, 335, 'PERSON'), (363, 375, 'PHONE')]}), ('Good afternoon Bilal,  Thank you for your message. We will close these work orders as soon as possible.  Best Wishes, Kerri SEA Campbell  NYU Client Services Center (212) 998-1001', {'entities': [(15, 20, 'PERSON'), (118, 136, 'PERSON'), (165, 179, 'PHONE')]}), ("Good afternoon Dustin,  We've reached out to the Carpentry shop manager for advisement. We will contact you as soon as we receive a response.  Please let us know if you have any other questions/concerns.  Best, Vanessa Morales Client Services Center 212-998-1001  Think green before printing this e-mail", {'entities': [(15, 21, 'PERSON'), (211, 226, 'PERSON'), (250, 262, 'PHONE')]}), ('Hello Vonetta,  Thanks for the chartfield information. We submitted work order *14-115000*. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001', {'entities': [(6, 13, 'PERSON'), (291, 303, 'PHONE'), (80, 89, 'REQUEST'), (253, 267, 'PERSON')]}), ('Good Afternoon Mitchell,  Thank you for contacting the Client Services Center.  The below work orders  have been submitted to the building staff for completion.  Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at http://www.nyu.edu/fcm.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 23, 'PERSON'), (227, 239, 'PHONE'), (254, 272, 'EMAIL'), (327, 334, 'PERSON'), (360, 372, 'PHONE')]}), ('Good afternoon Erica,  Thank you for contacting the Client Services Center. We submitted work order #:**14-129077 as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please feel free to contact us if you have any questions or concerns.  Kind regards, Eric Henry Emma Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (104, 113, 'REQUEST'), (312, 327, 'PERSON'), (351, 363, 'PHONE')]}), ('Good afternoon all,  Please see the below work order. The client requested to be contacted directly to schedule. He is hoping to have this estimate completed this week, if possible. Thank you!  *14-130261* - CENTRAL PLAZA,4 FL,STUDIO 1,PROVIDE ESTIMATE FOR INSTALLING OUTLETS AND REWIRING,SEE CLIENT FOR DETAILS,CLIENT REQUESTING ESTIMATE BY THU JULY 3  *Contact*    BILL MOULTEN *Contact Phone *   917-378-4774 *Contact Email *   wkm2@nyu.edu  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(195, 204, 'REQUEST'), (208, 235, 'FAC'), (367, 379, 'PERSON'), (342, 352, 'DATE'), (399, 411, 'PHONE'), (431, 443, 'EMAIL'), (452, 463, 'PERSON'), (491, 503, 'PHONE')]}), ('Good evening Montey,  Please be advised of the below work order generated for the Law School Operator Simon and sent to the HVAC shop.  *Work Order # 14-129862* *  Description:* VANDERBILT,2 FL,RM 206,TOO COLD,CLIENTS PRESENT  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(178, 200, 'FAC'), (13, 19, 'PERSON'), (102, 107, 'PERSON'), (308, 321, 'PERSON'), (345, 359, 'PHONE')]}), ('Good afternoon Peter,  Thank you for contacting the Client Services Center. We submitted work order #*14-129034* as per your request. Please refer to this number if you have any questions in the future regarding this matter.  *Description- *EDUCATION BLDG,3 FL,HALLWAY OUTSIDE RM 3YI TO 3Y4,PRIORITY TO PAINT AREA  Please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 20, 'PERSON'), (381, 396, 'PERSON'), (420, 432, 'PHONE'), (102, 111, 'REQUEST'), (241, 268, 'FAC')]}), ('Hello Orlando,  As per our conversation this morning I received a call from Barbara Backer asking us to cancel the work order below because her ac unit is now working.  Regards, Rachel Client Services Center', {'entities': [(76, 90, 'PERSON'), (178, 184, 'PERSON'), (6, 13, 'PERSON')]}), ('Good afternoon all,  Please see below. The tenant called to say that the doorman reported that Classic had visited her apartment yesterday and found all AC units to be working. She is upset and reports that although her AC has been set to 60 degrees for two hours, the room is still very hot. She grants PTE and would like to be scheduled for service as soon as possible.  *14-128546* - 3-WASH SQ V,5 FL,APT 5S,AC UNIT NOT COOLING PROPERLY,PTE URGENT  Contact    BARBARA BACKER Contact Phone    212 674 5984  Thank you for your help!  Best,  James Walsh NYU Client Services Center 212-998-1001', {'entities': [(374, 383, 'REQUEST'), (387, 410, 'FAC'), (463, 477, 'PERSON'), (495, 507, 'PHONE'), (542, 553, 'PERSON'), (581, 593, 'PHONE')]}), ('Good afternoon Patricia,  As per our phone conversation, this request will not be processed as you will have someone whom is related to the original item delivery make the necessary repairs within your own department. Should you require any additional assistance, please contact our office and we will help in any way we can.  Best,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(15, 23, 'PERSON'), (334, 349, 'PERSON'), (373, 385, 'PHONE')]}), ('Good Afternoon James,  WORK ORDER # 14-132942 has been updated with the below information.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 20, 'PERSON'), (36, 45, 'REQUEST'), (152, 159, 'PERSON'), (185, 197, 'PHONE')]}), ('Good afternoon Jim and Jeanette,  Thanks for continuing to follow up with us and we apologize for the discomfort.  We are very glad the client is going to be okay! I have updated the work order *14-132942* with all your notes and reached out to the HVAC front end.  They told me that a tech is en route to the building.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(15, 18, 'PERSON'), (23, 31, 'PERSON'), (195, 204, 'REQUEST'), (402, 415, 'PERSON')]}), ('Good afternoon Sharonda,  Thank you for contacting the Client Services Center. We have added a second phase to work order _14-132942_ as per your request and forward  to the HVAC shop.  Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  Enrique Salgado Client Services Center 212-998-1001   Think green before printing this e-mail', {'entities': [(347, 362, 'PERSON'), (386, 398, 'PHONE'), (15, 23, 'PERSON'), (123, 132, 'REQUEST')]}), ('Good evening Jeanette,  Thanks for following up with us and letting us know.  I have updated the work order *14-132942* accordingly.  Sharonda, please advise if this work order should be escalated.  Please feel free to contact us if you have any questions or concerns.  Regards,  Seamus Mullin Client Services Center (212) 998-1001', {'entities': [(13, 21, 'PERSON'), (134, 142, 'PERSON'), (109, 118, 'REQUEST'), (280, 293, 'PERSON'), (317, 331, 'PHONE')]}), ('Good evening Michael,  Please note that the below work orders have been forwarded to the HVAC system for completion and we have added these notes as well. We contacted the HVAC Shop as well to make them aware of these issues.  Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(13, 20, 'PERSON'), (341, 356, 'PERSON'), (380, 392, 'PHONE')]}), ('Good Afternoon Sharonda,  The below work orders have been updated but WORK ORDERS # 14-122748 and 14-119497 have been forwarded to the HVAC Shop for completion.  Please contact us with any questions or concerns.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(15, 23, 'PERSON'), (84, 93, 'REQUEST'), (98, 107, 'REQUEST'), (222, 229, 'PERSON'), (255, 267, 'PHONE')]}), ('Good afternoon Jasmina,  Thank you for contacting our office regarding this matter. We have sent an email to the client updating her on the status of this investigation and informing her that if she has any additional questions, she can contact our office for assistance. * ***Please contact our office if you have any questions or concerns regarding this matter.  Thank you for your time,  Tashay Gonzalez Client Services Center 212-998-1001', {'entities': [(391, 406, 'PERSON'), (15, 22, 'PERSON'), (430, 442, 'PHONE')]}), ("Good afternoon John,  Thank you for contacting the Client Services Center with this follow up. Please note we have forwarded this follow up to the Facility Manager for advisement, as last week the original request was evaluated and they sent to the AC Shop with their approval. The notes within the work order state that the chiller within the building was evaluated for operation, so it's possible the unit may have tripped again. Once we receive a response, we will contact you with the information provided and assist in resolving this issue in any way we can.  In the meantime, please let us know if you have any questions or concerns.  Best,  Tashay Gonzalez Client Services Center 212-998-1001", {'entities': [(15, 19, 'PERSON'), (648, 663, 'PERSON')]}), ('Hello John,  Thank you for contacting the Client Services Center. We submitted work order *14-125097* as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  Sonnett Smyrna Client Services Center 212-998-1001  Think green before printing this e-mail', {'entities': [(6, 10, 'PERSON'), (91, 100, 'REQUEST'), (284, 298, 'PERSON'), (322, 334, 'PHONE')]}), ('Good Afternoon Anya,  The keys you are requesting require prior approval from the facility manager.  Once approved, a work order will be generated for the below request.  Thanks,  Monique  Client Services Center  212-998-1001', {'entities': [(213, 225, 'PHONE'), (15, 19, 'PERSON'), (180, 187, 'PERSON')]}), ('Good Afternoon Kurt,  We have forwarded your request on to your supervisor for approval. We will let you know as soon as we receive a response.  In the mean time, please feel free to contact us with any questions or concerns.  Best,  Enrique E. Salgado Client Services Center 212-998-1001', {'entities': [(234, 252, 'PERSON'), (276, 288, 'PHONE'), (15, 19, 'PERSON')]}), ('Good Afternoon Jeffery,  Thank you for contacting the Client Services Center. We submitted work order #*14-130403* as per your request. Please refer to this number if you have any questions in the future regarding this matter.  Please let us know if you have any other questions/concerns.  Best,  *Enrique Salgado * *New York University **Supervisor, Client Services Center* contactcsc@nyu.edu  Think green before printing this e-mail', {'entities': [(15, 22, 'PERSON'), (298, 313, 'PERSON'), (375, 394, 'EMAIL'), (104, 113, 'REQUEST')]})]

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, new_model_name="animal", output_dir=None, n_iter=30):
    """Set up the pipeline and entity recognizer, and train the new entity."""
   
    random.seed(0)
    if model is not None:
       nlp = spacy.load(model)  # load existing spaCy model
       print("Loaded model '%s'" % model)
    else:
       nlp = spacy.blank("en")  # create blank Language class
       print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
     # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label('PERSON')  # add new entity label to entity recognizer
    ner.add_label('REQUEST')  # add new entity label to entity recognizer
    ner.add_label('DATE')  # add new entity label to entity recognizer
    ner.add_label('EMAIL')  # add new entity label to entity recognizer
    ner.add_label('FAC')  # add new entity label to entity recognizer
    ner.add_label('PHONE')  # add new entity label to entity recognizer
     # Adding extraneous labels shouldn't mess anything up
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "Do you like horses?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

main(model="en_core_web_lg",output_dir=r'C:\Users\Andrew\Documents\NYU IT\models', n_iter=50)



OSError: ignored

In [30]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: astor, keras-applications, six, termcolor, tensorboard, keras-preprocessing, tensorflow-estimator, numpy, grpcio, absl-py, protobuf, gast, wheel
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
# Load the model
nlp = spacy.load(r'C:\Users\Andrew\Documents\NYU IT\models')

In [5]:
#Test the model
string = '  Thank you Helga for contacting the Sean Nordquist Client Services Center at Seamus.  WORK ORDER # 14-130107  has been submitted to the building staff at West 4th St for completion. I would like the work to be done at 3 West 4th Road. Please feel free to contact us with any questions or concerns at 212-998-1001, via email at contactcsc@nyu.edu, or the Internet at'
doc = nlp(string)
print(doc.ents)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

(Seamus, 14-130107, 212-998-1001, contactcsc@nyu.edu)
Entities [('Seamus', 'PERSON'), ('14-130107', 'REQUEST'), ('212-998-1001', 'PHONE'), ('contactcsc@nyu.edu', 'EMAIL')]
